In [2]:
import os

In [3]:
%pwd

'd:\\Kidney-Disease-Classification-Project\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'd:\\Kidney-Disease-Classification-Project'

In [6]:
from dataclasses import dataclass
#dataclass: we used this so that can access the class from other files as well.
from pathlib import Path
#entity: it is the return type of a classes
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from src.CNNclassifier.constants import *
from src.CNNclassifier.utils.common import read_yaml,create_directories

In [17]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        #Creating an object of above class.
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
 


In [23]:
from src.CNNclassifier import logger
import gdown,zipfile

In [24]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function return None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-29 00:13:47,228: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-29 00:13:47,242: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-29 00:13:47,246: INFO: common: created directory at: artifacts]
[2023-12-29 00:13:47,248: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-29 00:13:47,251: INFO: 421001476: Downloading data from https://drive.google.com/file/d/1BmC6ljSoIcsCG7BPBk7pc1NrOM2WIMo6/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1BmC6ljSoIcsCG7BPBk7pc1NrOM2WIMo6
From (redirected): https://drive.google.com/uc?/export=download&id=1BmC6ljSoIcsCG7BPBk7pc1NrOM2WIMo6&confirm=t&uuid=bcb7883e-4d57-4fcb-8b27-daed75db8746
To: d:\Kidney-Disease-Classification-Project\artifacts\data_ingestion\data.zip
  1%|          | 8.39M/939M [00:18<55:47, 278kB/s] 

KeyboardInterrupt: 

  1%|          | 8.39M/939M [00:30<55:47, 278kB/s]